In [14]:
from ortools.linear_solver import pywraplp
from datetime import datetime

def assign_rooms_to_requests(rooms, requests, floor_weight=1, space_weight=1, reuse_weight=1):
    # Helper function to check if two time slots overlap
    def timeslot_overlap(ts1, ts2):
        # Convert string time slots to datetime objects
        start1, end1 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in ts1]
        start2, end2 = [datetime.strptime(ts, "%Y-%m-%d %H:%M") for ts in ts2]
        
        # Check if there is an overlap
        return not (end1 <= start2 or end2 <= start1)

    # Initialize the solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Room assignment variables
    room_vars = {}

    # Define valid rooms for each request
    valid_rooms_for_request = {}

    # Define room assignment variables based on request and time slot overlap
    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request['time_slots']):
                # Check if room meets request's capacity
                if request["type"] in room["type_capacity"] and room["type_capacity"][request["type"]] >= request["num_people"]:
                    # Define the room assignment variable for this request, room, and time slot
                    room_vars[(r_id, req_id, t_id)] = solver.BoolVar(f"room_{r_id}_req_{req_id}_time_{t_id}")
                    
                    # Collect valid rooms for the current request (based on capacity and time slot)
                    if req_id not in valid_rooms_for_request:
                        valid_rooms_for_request[req_id] = []
                    valid_rooms_for_request[req_id].append(room_vars[(r_id, req_id, t_id)])

    # Debugging: Print the valid rooms for each request
    for req_id in valid_rooms_for_request:
        valid_room_names = []
        for room_var in valid_rooms_for_request[req_id]:
            r_id, req_id, t_id = room_var.name().split('_')[1], room_var.name().split('_')[3], room_var.name().split('_')[5]
            valid_room_names.append(rooms[int(r_id)]["name"])  # Get room name based on id
        print(f"Available rooms for request {req_id}: {valid_room_names}")

    # Add constraint: each request can only be assigned one room at a time
    for req_id, valid_rooms in valid_rooms_for_request.items():
        solver.Add(solver.Sum(valid_rooms) == 1)

    # Add constraint: each room can only be assigned to one request at a time for overlapping time slots
    for r_id, room in enumerate(rooms):
        for req_id1, request1 in enumerate(requests):
            for t_id1, time_slot1 in enumerate(request1["time_slots"]):
                for req_id2, request2 in enumerate(requests):
                    if req_id1 != req_id2:
                        for t_id2, time_slot2 in enumerate(request2["time_slots"]):
                            if timeslot_overlap(time_slot1, time_slot2):
                                if (r_id, req_id1, t_id1) in room_vars and (r_id, req_id2, t_id2) in room_vars:
                                    solver.Add(room_vars[(r_id, req_id1, t_id1)] + room_vars[(r_id, req_id2, t_id2)] <= 1)

    # Define the floor distance, unused space, and reuse terms
    distance_terms = []
    unused_space_terms = []
    reuse_terms = []

    for r_id, room in enumerate(rooms):
        for req_id, request in enumerate(requests):
            for t_id, time_slot in enumerate(request['time_slots']):
                if (r_id, req_id, t_id) in room_vars:
                    # Calculate the floor distance as absolute difference between floors
                    for other_r_id, other_room in enumerate(rooms):
                        if other_r_id != r_id and (other_r_id, req_id, t_id) in room_vars:
                            floor_distance = abs(room["floor"] - other_room["floor"])
                            # Create an auxiliary variable for the product of two boolean variables
                            aux_var = solver.BoolVar(f"aux_{r_id}_{other_r_id}_{req_id}_{t_id}")
                            solver.Add(aux_var <= room_vars[(r_id, req_id, t_id)])
                            solver.Add(aux_var <= room_vars[(other_r_id, req_id, t_id)])
                            solver.Add(aux_var >= room_vars[(r_id, req_id, t_id)] + room_vars[(other_r_id, req_id, t_id)] - 1)
                            distance_terms.append(aux_var * floor_distance * floor_weight)

                    # Calculate unused space (room capacity - number of people)
                    unused_space = room["type_capacity"][request["type"]] - request["num_people"]
                    unused_space_terms.append(room_vars[(r_id, req_id, t_id)] * unused_space * space_weight)

                    # Add reuse term to favor reusing rooms
                    reuse_terms.append(room_vars[(r_id, req_id, t_id)] * reuse_weight)

    # Add an objective function to minimize the floor distance, unused space, and favor reusing rooms
    solver.Minimize(solver.Sum(distance_terms) + solver.Sum(unused_space_terms) - solver.Sum(reuse_terms))

    # Debug: Print the objective function terms before solving
    print("Objective function terms:")
    print("Floor distance terms:", distance_terms)
    print("Unused space terms:", unused_space_terms)
    print("Reuse terms:", reuse_terms)

    # Solve the problem
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print("Solution found:")
        for r_id, req_id, t_id in room_vars:
            if room_vars[(r_id, req_id, t_id)].solution_value() == 1:
                print(f"Request {req_id} assigned to Room {rooms[r_id]['name']} for time slot {requests[req_id]['time_slots'][t_id]}")
    else:
        print("No solution found.")

    # Debugging: Print the values of room_vars
#    for r_id, room in enumerate(rooms):
#        for req_id, request in enumerate(requests):
#            for t_id, time_slot in enumerate(request["time_slots"]):
#                if (r_id, req_id, t_id) in room_vars:
#                    print(f'room_vars[{r_id}, {req_id}, {t_id}] = {room_vars[(r_id, req_id, t_id)].solution_value()}')

# Example usage
import csv
rooms = []
with open('IHG_Montreal_Rooms.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        room = {}
        room["name"] = row["ROOM_NAME"]
        room["type_capacity"] = {key.split("_")[1]: int(row[key]) if row[key].isdigit() else 0 for key in row if key.startswith("CAPACITY_")}
        room["floor"] = int(row["FLOOR"]) if row["FLOOR"].isdigit() else 0
        rooms.append(room)
requests = [
    {"type": "THEATRE", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 13:00", "2023-10-01 15:00"]]},
    {"type": "THEATRE", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 14:00", "2023-10-01 15:00"]]},
    {"type": "RECEPTION", "num_rooms": 1, "num_people": 200, "time_slots": [["2023-10-01 15:00", "2023-10-01 19:00"]]},
]

# Call the function to solve the problem
assign_rooms_to_requests(rooms, requests, floor_weight=1, space_weight=0.5)

Available rooms for request 0: ['LES VOUTES', 'LA RUELLE DES FORTIFICATIONS', 'S. BERNHARDT', 'RAVEL', 'NORDHEIMER']
Available rooms for request 1: ['LES VOUTES', 'LA RUELLE DES FORTIFICATIONS', 'S. BERNHARDT', 'RAVEL', 'NORDHEIMER']
Available rooms for request 2: ['LES VOUTES', 'LA RUELLE DES FORTIFICATIONS', 'S. BERNHARDT', 'NORDHEIMER']
Objective function terms:
Floor distance terms: [<ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABEA87CD0>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABEA87C50>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABEB3BF10>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABE507F50>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABEBB40D0>, <ortools.linear_solver.python.linear_solver_natural_api.ProductCst object at 0x000002FABCC4D490>, <ortools.linear_solve

In [11]:
#Load the CSV IHG_Montreal_Rooms.csv and generate a list of Rooms from it
#each field CAPACITY_ will be a key in the dictionary type_capacity e.g. CAPACITY_THEATRE will be type_capacity["Theatre"]

import csv
rooms = []
with open('IHG_Montreal_Rooms.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        room = {}
        room["name"] = row["ROOM_NAME"]
        room["type_capacity"] = {key.split("_")[1]: int(row[key]) if row[key].isdigit() else 0 for key in row if key.startswith("CAPACITY_")}
        room["floor"] = int(row["FLOOR"]) if row["FLOOR"].isdigit() else 0
        rooms.append(room)



In [12]:
rooms

[{'name': 'LES VOUTES',
  'type_capacity': {'THEATRE': 80,
   'RECEPTION': 230,
   'BANQUET': 170,
   'CLASSROOM': 42,
   'BOARDROOM': 42,
   'USHAPE': 33,
   'HOLLOWSQUARE': 30,
   'CRESCENT': 36},
  'floor': 0},
 {'name': 'ARCHAMBAULT',
  'type_capacity': {'THEATRE': 60,
   'RECEPTION': 50,
   'BANQUET': 40,
   'CLASSROOM': 29,
   'BOARDROOM': 24,
   'USHAPE': 24,
   'HOLLOWSQUARE': 30,
   'CRESCENT': 24},
  'floor': 0},
 {'name': 'LA RUELLE DES FORTIFICATIONS',
  'type_capacity': {'THEATRE': 400,
   'RECEPTION': 800,
   'BANQUET': 330,
   'CLASSROOM': 0,
   'BOARDROOM': 0,
   'USHAPE': 0,
   'HOLLOWSQUARE': 0,
   'CRESCENT': 0},
  'floor': 0},
 {'name': 'LA GALERIE Dâ€™ART',
  'type_capacity': {'THEATRE': 50,
   'RECEPTION': 50,
   'BANQUET': 50,
   'CLASSROOM': 30,
   'BOARDROOM': 26,
   'USHAPE': 24,
   'HOLLOWSQUARE': 24,
   'CRESCENT': 24},
  'floor': 1},
 {'name': 'CHEZ PLUME',
  'type_capacity': {'THEATRE': 0,
   'RECEPTION': 100,
   'BANQUET': 80,
   'CLASSROOM': 0,
   'BOARD

In [6]:
rooms

[{'name': 'LES VOÛTES',
  'type_capacity': {'Theatre': 80, 'Reception': 230},
  'floor': 0},
 {'name': 'ARCHAMBAULT',
  'type_capacity': {'Theatre': 80, 'Reception': 50},
  'floor': 0},
 {'name': 'LA GALERIE D’ART',
  'type_capacity': {'Theatre': 100, 'Reception': 50},
  'floor': 1},
 {'name': 'CHEZ PLUME',
  'type_capacity': {'Theatre': 0, 'Reception': 100},
  'floor': 1},
 {'name': 'S. BERNHARDT ',
  'type_capacity': {'Theatre': 220, 'Reception': 320},
  'floor': 2}]